In [429]:
from kakao_get_location import *
from distance_matrix import compute_distance_osrm
from osrm_api import *

import pandas as pd 
import numpy as np 
import json
import pickle
import multiprocessing as mp
from multiprocessing import Pool
import warnings 

warnings.filterwarnings('ignore')

In [469]:
# 위경도 좌표 추가
items = pd.read_excel('./raw_data/제주박스_월별 배송주소(유산균).xlsx')
items  = get_location_dataframe(items)
items['번호'] = items['번호'].astype('int')
items['위도'] = items['위도'].astype('float')
items['경도'] = items['경도'].astype('float')

# 겹치는 주소 있기 때문에 일단 제거
# 추후에는 분할 결제를 고려하여 Volumns를 재 측정할 예정
items = items.drop_duplicates('전체주소')
items = items.reset_index(drop=True)
items['번호'] = range(1,len(items)+1)

100%|██████████| 348/348 [00:21<00:00, 15.86it/s]


In [470]:
warehouse_address = '제주특별자치도 제주시 화북일동 2131-8'

warehouse = get_location(warehouse_address)
warehouse = pd.DataFrame(warehouse).T
warehouse.columns = ['lat', 'lon']
warehouse['name'] = 'warehouse'

warehouse['lat'] = warehouse['lat'].astype('float')
warehouse['lon'] = warehouse['lon'].astype('float')

In [471]:
items_coords = items[['위도', '경도']]
items_coords.columns = ['lat', 'lon']

coords = pd.concat([warehouse[['lat','lon']],items_coords]).reset_index(drop=True)

---

### Optimization

In [5]:
# 데이터의 OD를 만들어 준다.(e.g. 데이터 수 : 200 -> 200 x 200
def convert_OD_dataframe(data):
    coords_np = data.values
    O = np.repeat(coords_np, len(coords_np), axis=0)
    D = np.tile(coords_np, reps=[len(coords_np),1])
    OD = np.hstack([O,D])

    OD = pd.DataFrame(OD)
    OD.columns = ['O_lat', 'O_lon', 'D_lat', 'D_lon']
    return OD

OD = convert_OD_dataframe(coords)

print(f'가용 할 수 있는 cpu :{mp.cpu_count()}')

# 병렬 처리에 사용할 프로세스 개수 지정
p = Pool(processes=30)

distance_matrix = p.map(compute_distance_osrm, OD.values)

length = len(coords)
distance_matrix = np.array(distance_matrix).reshape(length, length)

가용 할 수 있는 cpu :32


In [6]:
"""Simple Vehicles Routing Problem (VRP).

   This is a sample using the routing library python wrapper to solve a VRP
   problem.
   A description of the problem can be found here:
   http://en.wikipedia.org/wiki/Vehicle_routing_problem.

   Distances are in meters.
"""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model(distance_matrix, vehicles):
    data = {}
    # data['distance_matrix'] : 모든 좌표에 대한 거리를 계산한 거리 매트릭스
    data['distance_matrix'] = distance_matrix
    # data['num_vehicles'] : 운행하는 차량 수
    data['num_vehicles'] = vehicles
    # data['depot'] : 시작하는 위치의 인덱스 
    data['depot'] = 0
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))


def get_routes(solution, routing, manager):
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes

def main(distance_matrix, vehicles):
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model(distance_matrix, vehicles)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        140000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
        routes = get_routes(solution, routing, manager)
        return routes 
    else:
        print('No solution found !')

In [ ]:
routes_8 = main(distance_matrix.tolist(), 8)

In [ ]:
# route list 저장
route_data_file_name = 'routes_8_20220808'

with open(f"./route_data/{route_data_file_name}.pickle","wb") as fw:
    pickle.dump(routes_8, fw)

In [482]:
#route list load  
with open(f"./route_data/{route_data_file_name}.pickle","rb") as fr:
    routes_8 = pickle.load(fr)

- pulp 사용 
    - 시간과 편리성 비교

https://medium.com/jdsc-tech-blog/capacitated-vehicle-routing-problem-cvrp-with-python-pulp-and-google-maps-api-5a42dbb594c0

---

In [472]:
# items 데이터에 vehicle 배정과 배송 순서 부여
for idx, i in enumerate(routes_8):
    globals()[f'vehicle_{idx}'] = items.iloc[[j-1 for j in i[1:-1]]]
    globals()[f'vehicle_{idx}'] = globals()[f'vehicle_{idx}'].reset_index(drop=True)
    globals()[f'vehicle_{idx}']['delivery_order'] = range(len(globals()[f'vehicle_{idx}']))
    globals()[f'vehicle_{idx}']['vehicle_id'] = idx
    
items = pd.concat([globals()[f'vehicle_{idx}'] for idx,_ in enumerate(routes_8)])

In [473]:
items = items.sort_values('번호') 
items = items.reset_index(drop=True) 

geometry_dict = {idx+1:[i[0],i[1]] for idx, i in enumerate(items[['경도','위도']].values)}
geometry_dict[0] = [warehouse['lon'][0], warehouse['lat'][0]]

def convert_routes_point_geometry(geometry_dict, routes):
    routesgeometry = []
    for i in tqdm(range(len(routes))):
        sub_routes = [geometry_dict[j] for j in routes[i]]
        routesgeometry.append(sub_routes)
    return routesgeometry

routes_geometry = convert_routes_point_geometry(geometry_dict, routes_8)

100%|██████████| 8/8 [00:00<00:00, 98112.37it/s]


In [475]:
for idx,i in enumerate(routes_geometry):
    delivery_routes = np.array(i)
    delivery_routes = np.hstack([delivery_routes[:-1,:],delivery_routes[1:,:]])

    globals()[f'vehicle_{idx}'] = [get_res(i) for i in delivery_routes]

    vehicle_timestamps = [get_total_timestamp(i) for i in globals()[f'vehicle_{idx}']]
    vehicle_routes = [get_total_route(i) for i in globals()[f'vehicle_{idx}']]
    vehicle_distance = [get_distance(i) for i in globals()[f'vehicle_{idx}']]
    vehicle_duration = [get_duration(i) for i in globals()[f'vehicle_{idx}']]

    globals()[f'items_vehicle_{idx}']= items.loc[items["vehicle_id"] == idx]
    globals()[f'items_vehicle_{idx}'] = globals()[f'items_vehicle_{idx}'].sort_values('delivery_order')

    globals()[f'vehicle_{idx}_timestamps_last'] = vehicle_timestamps[-1]
    globals()[f'vehicle_{idx}_routes_last']= vehicle_routes[-1]
    globals()[f'vehicle_{idx}_distance_last']= vehicle_distance[-1]
    globals()[f'vehicle_{idx}_duration_last']= vehicle_duration[-1]

    globals()[f'items_vehicle_{idx}']['timestamps'] = vehicle_timestamps[:-1] 
    globals()[f'items_vehicle_{idx}']['routes'] = vehicle_routes[:-1] 
    globals()[f'items_vehicle_{idx}']['distance'] = vehicle_distance[:-1] 
    globals()[f'items_vehicle_{idx}']['duration'] = vehicle_duration[:-1] 
    
    # 현재 아파트 동 단위 좌표가 동일함, 일단 고려 하지 않고 제거 후 구현
    globals()[f'items_vehicle_{idx}'] = globals()[f'items_vehicle_{idx}'].loc[globals()[f'items_vehicle_{idx}']['duration'] != 0] 
    globals()[f'items_vehicle_{idx}']['delivery_order'] = range(len(globals()[f'items_vehicle_{idx}']))
    
items = pd.concat([globals()[f'items_vehicle_{i}'] for i in range(len(routes_geometry))])

In [476]:
all_vehicle_trip = []  # 배송 차량 정보
items_information = [] # 상품 정보

for idx in tqdm(range(len(routes_geometry))):
    # vehicle items 수
    globals()[f'vehicle_{idx}_items_cnt'] = len(globals()[f'items_vehicle_{idx}'])
    
    ### timestampe 9시부터 적용 및 배송 시간 5분으로 추가하여 timestamp 재정의
    # 배송 시간 누적합으로 배송 시간 재정의
    timestamp = globals()[f'items_vehicle_{idx}']['timestamps'].values.tolist()
    timestamp = np.array([t[-1] for t in timestamp])
    timestamp = np.cumsum(timestamp)
    
    # np.cumsum(np.repeat(5, len(a[:-1]))) # 배송시간 각 5분으로 잡음. 
    timestamp = timestamp[:-1] + np.cumsum(np.repeat(5 ,len(timestamp[:-1])))
    timestamp = np.hstack([0,timestamp])
    # 배송 시작 시간 9시(540분)으로 정의
    timestamp = timestamp + 540 # 540 (9시)
    # 배송 시간 적용
    timestamp = [t+increase_t for t, increase_t in zip(globals()[f'items_vehicle_{idx}']['timestamps'].values, timestamp)]
    # 마지막 배송 시간 따로 정의
    globals()[f'vehicle_{idx}_timestamps_last'] = np.array(globals()[f'vehicle_{idx}_timestamps_last'])  + (timestamp[-1][-1] + 5)
    timestamp.append(globals()[f'vehicle_{idx}_timestamps_last'])
    
    
    vehicle_trips = []
    # items 배송하로 가는 information
    for j in range(len(globals()[f'items_vehicle_{idx}'])):

        vehicle = dict() 

        vehicle['vehicle_id'] = idx 
        vehicle['to_client'] = globals()[f'items_vehicle_{idx}']['번호'].tolist()[j]
        vehicle['items_cnt'] = globals()[f'vehicle_{idx}_items_cnt']
        vehicle['trips'] = globals()[f'items_vehicle_{idx}']['routes'].tolist()[j]
        vehicle['timestamps'] = timestamp[j].tolist()

        globals()[f'vehicle_{idx}_items_cnt'] -= 1
        vehicle_trips.append(vehicle)
        
        # update items_information
        items_inf = dict()
        items_inf['item_id'] = globals()[f'items_vehicle_{idx}']['번호'].tolist()[j]
        items_inf['path'] = globals()[f'items_vehicle_{idx}']['routes'].tolist()[j][-1]
        items_inf['timestamp'] = [540,timestamp[j].tolist()[-1]]
        items_information.append(items_inf)

    
    # 최종 차고지로 오는 information    
    vehicle['vehicle_id'] = idx
    vehicle['to_client'] = 0
    vehicle['items_cnt'] = 0
    vehicle['trips'] = globals()[f'vehicle_{idx}_routes_last']
    vehicle['timestamps'] = timestamp[-1].tolist()
    vehicle_trips.append(vehicle)
    
    all_vehicle_trip.extend(vehicle_trips)

100%|██████████| 8/8 [00:00<00:00, 1434.44it/s]


In [479]:
with open(f'./result_data/items_information.json', 'w') as f:
    json.dump(items_information,f)
    
with open(f'./result_data/jeju_delivery_trip_8.json', 'w') as f:
    json.dump(all_vehicle_trip,f)